In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The purpose of this notebook is to visualize IPL data and make intuitions from it

# **1)Loading the dataset**

In [ ]:
df=pd.read_csv('/kaggle/input/iplcricket-league-dataset-for-beginner-analaysis/IPL_final.csv')

In [ ]:
df.head() #Top 5 rows

In [ ]:
df.columns #columns present in the dataset

In [ ]:
len(df.columns)

As we may observe, there are 14 columns in  our dataset out of which 2 columns are of no use to us. i.e. the unnamed columns.

# **2)Importing Matplotlib.pyplot and seaborn**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 3)Making sense of the data

In [ ]:
df.info()

As we can see, 11 of the useful columns are of the type object which have to be converted into int type while feeding it to any sort of machine learning model

# 4)Encoding the data


In [ ]:
print(type(df['Team1'][0]))

In [ ]:
l=[]
for i in df['Team1']:
    l.append(i.strip())
df['Team1']=l
l=[]
for i in df['Team2']:
    l.append(i.strip())
df['Team2']=l #removing the extra spaces

In [ ]:
l1=df['Team1'].unique()
l2=df['Team2'].unique()

In [ ]:
en_l1={}
x=0
for i in l1:
    en_l1[i]=x
    x=x+1      

These are the encoded values

In [ ]:
en_l1

In [ ]:
df['Team1']=df['Team1'].map(en_l1)
df['Team2']=df['Team2'].map(en_l1)

In [ ]:
df.head(10)

In [ ]:
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)#dropping unnamed columns

In [ ]:
l=[]
lis=df['Toss'].unique()
for i in lis:
    l.append(i)
d_l={}
x=0
for i in l:
    s=i[0]
    if(s=='8' or s=='4' or s=='.'):
        d_l[i]=int(10000)
    else:
        d_l[i]=int(x)
        x=x+1

We may notice that there are few rows such as '8:00PM' and '4:00 PM' ,we'll have to handel these cases by giving a large value

In [ ]:
df['Toss']=df['Toss'].map(d_l)
d_l


Now we have encoded the team names and removed unnamed colums from out dataframe

In [ ]:
l=df['Place'].unique()
x=0
en_pl={}
for i in l:
    en_pl[i]=x
    x=x+1
df['Place']=df['Place'].map(en_pl)


In [ ]:
df.head()

We have TossDecision,Result,Tied,won_runs and won_wickets left to encode.

In [ ]:
lis=df['Tied'].unique()
print("Match" in lis[1])

In [ ]:
d={}
for i in lis:
    if(type(i)== float):
        d[i]=0
    elif ("Match abandoned" in i):
        d[i]=1
    elif("Match tied" in i):
        d[i]=2
    


In [ ]:
df['Tied']=df['Tied'].map(d)

In [ ]:
df['won_runs'].unique()

In [ ]:
lis=df['won_runs'].unique()
d={}
for i in lis:
    if(type(i)==float):
        d[i]=0
    else:
        d[i]=int(i[0:3])
df['won_runs']=df['won_runs'].map(d)

In [ ]:
lis=df['won_wickets'].unique()
d={}
for i in lis:
    if(type(i)==float):
        d[i]=0
    else:
        d[i]=int(i[0:3])
df['won_wickets']=df['won_wickets'].map(d)

In [ ]:
lis=df['Result'].unique()
l=[]
for r in lis:
    l.append(r.split(" won")[0])
s=[]
for i in df['Result']:
    s.append(i.split(" won")[0].lower())
df['Result']=s      
df.head()

In [ ]:
en_l={}
x=0
for i in l1:
    en_l[i.lower()]=x
    x=x+1
for i in l:
    if ("Match abandoned" in i):
        en_l[i]=20
    elif("Match tied" in i):
        en_l[i]=21
   
df['Result']=df['Result'].map(en_l)  

In [ ]:
en_l

In [ ]:
en_l1

In [ ]:
df.head()

In [ ]:
df.info()

As you can see ,we have converted all the useful columns into int or float by encoding except Toss and TossDecision is still of the type object 

In [ ]:
dec={'bat':1,'bowl':2}
df['TossDecision']=df['TossDecision'].map(dec)

In [ ]:
df.head()

We have finished encoding all the required information

# Data Visualisation

Let's see which team played the most number of matches. We have 2 rows for teams - Team1 and Team2

In [ ]:
df['Team1'].value_counts().sort_index()

In [ ]:
df['Team2'].value_counts().sort_index()

We'll have to add the two 

In [ ]:
teams=df['Team1'].value_counts().sort_index()+df['Team2'].value_counts().sort_index()

In [ ]:
plt.figure(figsize=(10,10))
plt.bar([0,1,2,3,4,5,6,7,8,9,10,11,12],teams,color='g',width=0.75)

In [ ]:
en_l1

From this we may notice that 4)Mumbai Indians have played the most number of matches, but we may notice that Pune Warriors and Rising Pune Supergiant are the same team.I haven't taken that into account

Let's now see which year had most number of matches

In [ ]:
year=df['Year'].value_counts().sort_index()
plt.figure(figsize=(10,10))
plt.pie(year,labels=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019],autopct='%1.2f%%')

2012 and 2013 had the most number of mathes and, 2009 and 2008 had the least number of matches scheduled.

In [ ]:
dec=df['TossDecision'].value_counts().sort_index()
plt.figure(figsize=(10,10))
plt.bar(['Bat','bowl'],dec)

Almost 50% of the teams that won the toss decided to bowl first!

Finally, let's use seaborn to plot a heatmap to determine the relation between different columns

In [ ]:
sns.heatmap(df.corr(),vmax=1,annot=True,fmt=".2f")

For our dataset, there isn't much inference that can be made from the heatmap 
The negative values depicts inverse relation whereas positive values depict direct values.

This is just an introduction to basic data cleaning and data visualization a lot of things can be made with the resourses available.
I hope this notebook was useful and you can use it as a reference for your future projects

**Adios amigos!**